In [3]:
import os
import sys

# Clone or pull part
repo_url = "https://github.com/fraco03/6D_pose.git"
repo_dir = "/content/6D_pose"
branch = "pose_rgb"

if not os.path.exists(repo_dir):
    !git clone -b {branch} {repo_url}
    print(f"Cloned {repo_url}")
else:
    %cd {repo_dir}
    !git fetch origin
    !git checkout {branch}
    !git reset --hard origin/{branch}
    %cd ..
    print(f"Updated {repo_url}")

if repo_dir not in sys.path:
    sys.path.insert(0, repo_dir)

%cd 6D_pose

/content/6D_pose
Already on 'pose_rgb'
Your branch is up to date with 'origin/pose_rgb'.
HEAD is now at 5a413ca Fix: minor changes
/content
Updated https://github.com/fraco03/6D_pose.git
/content/6D_pose


In [ ]:
%cd ..
!gdown --fuzzy https://drive.google.com/file/d/1zNthSyiBdPUfn7BmUKPbKoGgQdG1vGnS/view?usp=drive_link -O Linemod_preprocessed.zip
!unzip Linemod_preprocessed.zip
%cd 6D_pose

In [4]:
from google.colab import drive
from utils.load_data import mount_drive

mount_drive()

# dataset_root = "/content/drive/MyDrive/Linemod_preprocessed"
dataset_root = "/content/Linemod_preprocessed"
print(f"\n✅ Setup complete!")
print(f"📁 Dataset path: {dataset_root}")

Mounted at /content/drive
✅ Drive mounted at /content/drive

✅ Setup complete!
📁 Dataset path: /content/Linemod_preprocessed


In [5]:
!pip install plyfile

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.3/43.3 kB 3.1 MB/s eta 0:00:00


In [6]:
from src.pose_pointnet.pointcloud_dataset import LineModPointCloudDataset
from src.pose_pointnet.pointnet_model import PointNetPose
from src.pose_rgb.loss import AutomaticWeightedLoss
from torch.utils.data import DataLoader
import torch
from tqdm import tqdm

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"🔥 Using device: {DEVICE}")

🔥 Using device: cuda


In [7]:
# Crea dataset con point clouds
# Nota: Prediciamo SOLO rotazione, la traslazione viene ricavata da depth + bbox usando pinhole geometry

train_dataset = LineModPointCloudDataset(
    root_dir=dataset_root,
    split='train',
    num_points=512,
    use_rgb=True
)

test_dataset = LineModPointCloudDataset(
    root_dir=dataset_root,
    split='test',
    num_points=512,
    use_rgb=True
)

print(f"Train samples: {len(train_dataset)}")
print(f"Test samples: {len(test_dataset)}")

✅ LineModConfig initialized: /content/Linemod_preprocessed
🔄 Preloading YAML files...
✅ Preloaded YAML data for 13 objects
📊 Loaded 2373 samples for train split
🔄 Preloading YAML files...
✅ Preloaded YAML data for 13 objects
📊 Loaded 13407 samples for test split
Train samples: 2373
Test samples: 13407


In [8]:
# Visualizza un sample
sample = train_dataset[0]

print("Sample keys:", sample.keys())
print(f"Point cloud shape: {sample['point_cloud'].shape}")  # (512, 6)
print(f"Rotation shape: {sample['rotation'].shape}")        # (4,) - TENSOR
print(f"Depth Z: {sample['depth_z']:.4f} m")
print(f"\nRotation (quat, tensor): {sample['rotation']}")
print(f"Bbox info (tensor): {sample['bbox_info']}")

Sample keys: dict_keys(['point_cloud', 'bbox_info', 'rotation', 'object_id', 'img_id', 'cam_K', 'bbox', 'depth_z', 'gt_translation'])
Point cloud shape: torch.Size([512, 6])
Rotation shape: torch.Size([4])
Depth Z: 1.0280 m

Rotation (quat, tensor): tensor([ 0.3326,  0.6473,  0.6364, -0.2555])
Bbox info (tensor): tensor([0.4180, 0.4563, 0.0672, 0.1167])


In [9]:
# DataLoaders - Ottimizzati per performance
train_loader = DataLoader(
    train_dataset,
    batch_size=64,  # Aumentato da 32 per migliore GPU utilization
    shuffle=True,
    num_workers=2,  # Aumentato da 2
    pin_memory=True  # Velocizza transfer CPU->GPU
)

test_loader = DataLoader(
    test_dataset,
    batch_size=64,
    shuffle=False,
    num_workers=2,
    pin_memory=True
)

In [10]:
# Import visualization utilities
from utils.projection_utils import visualize_pose_comparison
from utils.linemod_config import get_linemod_config
from utils.projection_utils import setup_projection_utils
import matplotlib.pyplot as plt
import cv2
import numpy as np
from pathlib import Path

# Config per caricare modelli 3D
config = get_linemod_config(dataset_root)
setup_projection_utils(dataset_root)

def compute_translation_from_depth(bbox_info, depth_z, cam_K, img_shape):
    """
    Compute 3D translation using pinhole geometry.
    bbox_info: tensor or numpy array
    depth_z: scalar (tensor or float)
    cam_K: tensor or numpy array
    """
    H, W = img_shape

    # Convert to numpy if needed
    if isinstance(bbox_info, torch.Tensor):
        bbox_info = bbox_info.numpy()
    if isinstance(cam_K, torch.Tensor):
        cam_K = cam_K.numpy()
    if isinstance(depth_z, torch.Tensor):
        depth_z = depth_z.item()

    cx_norm, cy_norm = bbox_info[0], bbox_info[1]

    # Convert normalized to pixel coordinates
    cx_pixel = cx_norm * W
    cy_pixel = cy_norm * H

    # Back-project using pinhole model
    fx = cam_K[0, 0]
    fy = cam_K[1, 1]
    cx_intr = cam_K[0, 2]
    cy_intr = cam_K[1, 2]

    Z = depth_z
    X = (cx_pixel - cx_intr) * Z / fx
    Y = (cy_pixel - cy_intr) * Z / fy

    return np.array([X, Y, Z])

def visualize_training_sample(model, dataset, dataset_root, checkpoint_dir, epoch, device='cuda'):
    """
    Visualizza una predizione casuale e la salva.
    """
    import shutil

    model.eval()

    # Sample casuale
    idx = np.random.randint(0, len(dataset))
    sample = dataset[idx]

    obj_id = sample['object_id']
    img_id = sample['img_id']

    # Carica immagine originale
    img_path = Path(dataset_root) / "data" / f"{obj_id:02d}" / "rgb" / f"{img_id:04d}.png"
    image = cv2.imread(str(img_path))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    H, W = image.shape[:2]

    # Prepare input
    point_cloud = sample['point_cloud'].unsqueeze(0).to(device)
    bbox_info = sample['bbox_info'].unsqueeze(0).to(device)

    # Inference - ONLY rotation
    with torch.no_grad():
        pred_rot = model(point_cloud, bbox_info)

    # Convert to numpy
    pred_rot = pred_rot.squeeze(0).cpu().numpy()
    gt_rot = sample['rotation'].cpu().numpy()
    cam_K = sample['cam_K'].numpy()

    # Compute translations from depth + bbox (same for both gt and pred)
    trans = compute_translation_from_depth(sample['bbox_info'], sample['depth_z'], cam_K, (H, W))

    # Visualizza (rotation is different, translation is same)
    img_vis = visualize_pose_comparison(
        image, obj_id, cam_K,
        gt_rot, trans,
        pred_rot, trans
    )

    # Calcola errore di rotazione
    dot_product = np.abs(np.dot(pred_rot, gt_rot))
    dot_product = np.clip(dot_product, -1.0, 1.0)
    angle_error = 2 * np.arccos(dot_product) * 180 / np.pi

    # Salva
    tmp_path = f"/tmp/vis_epoch_{epoch:03d}_obj_{obj_id:02d}.png"
    fig = plt.figure(figsize=(12, 10))
    plt.imshow(img_vis)
    plt.axis('off')
    plt.title(f"Object {obj_id} | Rotation Error: {angle_error:.2f}°")
    plt.savefig(tmp_path, bbox_inches='tight', dpi=100)
    plt.close()

    # Copia su Drive
    vis_dir = Path(checkpoint_dir) / "visualizations"
    vis_dir.mkdir(exist_ok=True)
    drive_path = vis_dir / f"epoch_{epoch:03d}_obj_{obj_id:02d}.png"
    shutil.copy(tmp_path, str(drive_path))

    print(f"✅ Saved: epoch {epoch} | Rotation Error: {angle_error:.2f}°")

    model.train()


In [16]:
import os
import json
from datetime import datetime
from itertools import islice
import matplotlib.pyplot as plt
from torch.amp import autocast, GradScaler
from src.pose_pointnet.pointnet_model import PointNetPose


# ==========================================
# HYPERPARAMETERS
# ==========================================
LEARNING_RATE = 1e-4
NUM_EPOCHS = 50
USE_MIXED_PRECISION = True
NUM_MODEL_POINTS = 1024  # Sample fixed number from each model

timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
CHECKPOINT_DIR = f'/content/drive/MyDrive/runs/pointnet_{timestamp}'
os.makedirs(CHECKPOINT_DIR, exist_ok=True)

# Initialize PointNet Model - ROTATION ONLY
model = PointNetPose(input_channels=6, use_batch_norm=True).to(DEVICE)

# Loss - SIMPLE: Only Rotation Loss
from src.pose_pointnet.loss import PoseMatchingLoss
criterion = PoseMatchingLoss()

# ==========================================
# PRECACHE MODEL POINTS (ONE TIME)
# ==========================================
print(f"⚡ Precaching model_points (sampling {NUM_MODEL_POINTS} points per model)...")
model_points_gpu = {}
for obj_id in range(1, 16):  # Objects 01-15
    try:
        vertices = config.get_model_3d(obj_id)  # (N, 3) in mm
        vertices_m = vertices / 1000.0  # Convert to meters

        # Sample fixed number of points
        if vertices_m.shape[0] > NUM_MODEL_POINTS:
            # Downsample: random sampling without replacement
            indices = np.random.choice(vertices_m.shape[0], NUM_MODEL_POINTS, replace=False)
            vertices_m = vertices_m[indices]
        elif vertices_m.shape[0] < NUM_MODEL_POINTS:
            # Upsample: random sampling with replacement
            indices = np.random.choice(vertices_m.shape[0], NUM_MODEL_POINTS, replace=True)
            vertices_m = vertices_m[indices]

        # To GPU
        model_points_gpu[obj_id] = torch.from_numpy(vertices_m).float().to(DEVICE)
    except Exception as e:
        print(f"⚠️  Could not load model points for object {obj_id}: {e}")

print(f"✅ Cached {len(model_points_gpu)} object models ({NUM_MODEL_POINTS} points each) to {DEVICE}")

# Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

# Mixed Precision Scaler
scaler = GradScaler('cuda') if USE_MIXED_PRECISION else None

train_losses = []
val_losses = []
best_val_loss = float('inf')
best_epoch = 0

print(f"\n🔥 STARTING POINTNET ROTATION-ONLY TRAINING on {DEVICE}...")
print(f"📁 Checkpoints: {CHECKPOINT_DIR}")
print(f"⚙️  Loss: PoseMatchingLoss (quaternion geodesic + point transformation)")
print(f"⚡ Mixed Precision: {USE_MIXED_PRECISION}")

⚡ Precaching model_points (sampling 1024 points per model)...
✅ Cached 15 object models (1024 points each) to cuda

🔥 STARTING POINTNET ROTATION-ONLY TRAINING on cuda...
📁 Checkpoints: /content/drive/MyDrive/runs/pointnet_20251218_134341
⚙️  Loss: PoseMatchingLoss (quaternion geodesic + point transformation)
⚡ Mixed Precision: True


## 🎯 Rotation-Only Model Architecture

**Key Insight**: With depth available, we only need to predict **rotation**. Translation is computed directly using pinhole geometry:

$$X = \frac{(c_x - c_x^{intr}) \cdot Z}{f_x}, \quad Y = \frac{(c_y - c_y^{intr}) \cdot Z}{f_y}, \quad Z = \text{depth}[c_x, c_y]$$

**Advantages:**
- Simpler model (no translation head)
- Faster training (only rotation loss)
- More stable (depth provides ground truth Z)
- Cleaner optimization (one prediction target instead of two)

**Dataset returns:**
- `point_cloud`: Local point cloud (N×3 or N×6 with RGB)
- `rotation`: Target quaternion
- `depth_z`: Z coordinate from depth at bbox center (for inference)

In [18]:
# ==========================================
# TRAINING LOOP - POSE MATCHING LOSS
# ==========================================
for epoch in range(NUM_EPOCHS):

    # --- TRAIN PHASE ---
    model.train()
    running_train_loss = 0.0

    pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{NUM_EPOCHS} [Train]")

    for batch in pbar:
        # Move ENTIRE batch to device
        batch_cuda = {
            'point_cloud': batch['point_cloud'].to(DEVICE, non_blocking=True),
            'bbox_info': batch['bbox_info'].to(DEVICE, non_blocking=True),
            'rotation': batch['rotation'].to(DEVICE, non_blocking=True),
            'object_id': batch['object_id'],
        }

        # Get model_points from precached GPU cache
        obj_ids = batch_cuda['object_id']
        model_points_batch = torch.stack([model_points_gpu[obj_id.item()] for obj_id in obj_ids])

        optimizer.zero_grad()

        # Forward
        if USE_MIXED_PRECISION:
            with autocast('cuda'):
                pred_rot = model(batch_cuda['point_cloud'], batch_cuda['bbox_info'])
                loss = criterion(pred_rot, batch_cuda['rotation'], model_points_batch)

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
        else:
            pred_rot = model(batch_cuda['point_cloud'], batch_cuda['bbox_info'])
            loss = criterion(pred_rot, batch_cuda['rotation'], model_points_batch)
            loss.backward()
            optimizer.step()

        running_train_loss += loss.item()
        pbar.set_postfix({'Loss': f"{loss.item():.4f}"})

    avg_train_loss = running_train_loss / len(train_loader)
    train_losses.append(avg_train_loss)

    # --- VALIDATION PHASE ---
    model.eval()
    running_val_loss = 0.0
    val_batches_limit = 50
    count_batches = 0

    with torch.no_grad():
        val_iterator = islice(test_loader, val_batches_limit)
        val_pbar = tqdm(val_iterator, total=val_batches_limit, desc="Validating")

        for batch in val_pbar:
            # Move ENTIRE batch to device
            batch_cuda = {
                'point_cloud': batch['point_cloud'].to(DEVICE, non_blocking=True),
                'bbox_info': batch['bbox_info'].to(DEVICE, non_blocking=True),
                'rotation': batch['rotation'].to(DEVICE, non_blocking=True),
                'object_id': batch['object_id'],
            }


            # Get model_points from precached GPU cache
            obj_ids = batch_cuda['object_id']
            model_points_batch = torch.stack([model_points_gpu[obj_id.item()] for obj_id in obj_ids])

            if USE_MIXED_PRECISION:
                with autocast('cuda'):
                    pred_rot = model(batch_cuda['point_cloud'], batch_cuda['bbox_info'])
                    loss = criterion(pred_rot, batch_cuda['rotation'], model_points_batch)
            else:
                pred_rot = model(batch_cuda['point_cloud'], batch_cuda['bbox_info'])
                loss = criterion(pred_rot, batch_cuda['rotation'], model_points_batch)

            running_val_loss += loss.item()
            count_batches += 1

    avg_val_loss = running_val_loss / count_batches if count_batches > 0 else 0.0
    val_losses.append(avg_val_loss)

    # --- REPORT & SAVE ---
    print(f"📊 Epoch {epoch+1}: Train Loss={avg_train_loss:.4f} | Val Loss={avg_val_loss:.4f}")

    # --- VISUALIZE RANDOM SAMPLE ---
    if (epoch + 1) % 3 == 0:
        print(f"🎨 Visualizing random validation sample...")
        visualize_training_sample(model, test_dataset, dataset_root, CHECKPOINT_DIR, epoch, DEVICE)

    # Save best model
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        best_epoch = epoch + 1

        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'val_loss': best_val_loss
        }, os.path.join(CHECKPOINT_DIR, "best_model.pth"))

        print(f"🏆 New Best Model! (Loss: {best_val_loss:.4f})")

    # Save last checkpoint
    if (epoch + 1) == NUM_EPOCHS:
        torch.save({
            'epoch': epoch+1,
            'model_state_dict': model.state_dict(),
        }, os.path.join(CHECKPOINT_DIR, f"checkpoint_ep{epoch+1}.pth"))

print("\n🎉 TRAINING COMPLETE!")

Validating: 100%|██████████| 50/50 [00:49<00:00,  1.01it/s]


📊 Epoch 1: Train Loss=0.0567 | Val Loss=0.0475
🏆 New Best Model! (Loss: 0.0475)


Validating: 100%|██████████| 50/50 [00:45<00:00,  1.10it/s]


📊 Epoch 2: Train Loss=0.0558 | Val Loss=0.0469
🏆 New Best Model! (Loss: 0.0469)


Validating: 100%|██████████| 50/50 [00:44<00:00,  1.11it/s]


...

📊 Epoch 23: Train Loss=0.0484 | Val Loss=0.0453


Validating: 100%|██████████| 50/50 [00:45<00:00,  1.10it/s]


📊 Epoch 24: Train Loss=0.0490 | Val Loss=0.0441
🎨 Visualizing random validation sample...
✅ Saved: epoch 23 | Rotation Error: 86.59°


Epoch 25/50 [Train]:  68%|██████▊   | 26/38 [00:25<00:11,  1.02it/s, Loss=0.0514]


KeyboardInterrupt: 

In [ ]:
# Plot training history
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.plot(train_losses, label='Train Loss', marker='o', alpha=0.7)
plt.plot(val_losses, label='Validation Loss', marker='s', alpha=0.7)
if best_epoch > 0:
    plt.axvline(x=best_epoch-1, color='r', linestyle='--', alpha=0.5,
                label=f'Best Epoch ({best_epoch})')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('PointNet Training History')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.plot(train_losses, label='Train Loss', marker='o', alpha=0.7)
plt.plot(val_losses, label='Validation Loss', marker='s', alpha=0.7)
if best_epoch > 0:
    plt.axvline(x=best_epoch-1, color='r', linestyle='--', alpha=0.5,
                label=f'Best Epoch ({best_epoch})')
plt.xlabel('Epoch')
plt.ylabel('Loss (log scale)')
plt.yscale('log')
plt.title('PointNet Training History (Log)')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(os.path.join(CHECKPOINT_DIR, 'training_history.png'), dpi=150)
plt.show()

print(f"\n📊 Training Statistics:")
print(f"   Best epoch: {best_epoch}")
print(f"   Best val loss: {best_val_loss:.6f}")
print(f"   Final train loss: {train_losses[-1]:.6f}")

# Save history
history = {
    'train_losses': [float(x) for x in train_losses],
    'val_losses': [float(x) for x in val_losses],
    'best_epoch': int(best_epoch),
    'best_val_loss': float(best_val_loss),
    'timestamp': timestamp
}

with open(os.path.join(CHECKPOINT_DIR, 'history.json'), 'w') as f:
    json.dump(history, f, indent=2)

## Visualize Point Cloud Sample

Visualizing point cloud from random point in the dataset

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Choose sample
sample = train_dataset[100]
pc = sample['point_cloud'].numpy()  # (512, 6) [x, y, z, r, g, b]

fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

# Plot 3D points with RGB colors
ax.scatter(pc[:, 0], pc[:, 1], pc[:, 2],
           c=pc[:, 3:6], s=2, alpha=0.6)

ax.set_xlabel('X (m)')
ax.set_ylabel('Y (m)')
ax.set_zlabel('Z (m)')
ax.set_title(f'Point Cloud - Object {sample["object_id"]}')
plt.show()

print(f"Object ID: {sample['object_id']}")
print(f"Rotation (quat, tensor): {sample['rotation']}")
print(f"GT Translation (m, tensor): {sample['gt_translation']}")

In [21]:
# ==========================================
# QUICK EVALUATION - ADD/ADD-S METRICS
# ==========================================

# Load best model
checkpoint = torch.load(os.path.join(CHECKPOINT_DIR, 'best_model.pth'))
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

# Evaluate on test set
from metrics.ADD_metric import compute_ADD_metric_quaternion
results = []

with torch.no_grad():
    for i in tqdm(range(100)):  # 100 samples
        sample = test_dataset[i]
        pred_rot = model(sample['point_cloud'].unsqueeze(0).to(DEVICE),
                        sample['bbox_info'].unsqueeze(0).to(DEVICE))

        model_3d = config.get_model_3d(sample['object_id'], unit='m')
        add = compute_ADD_metric_quaternion(
            model_3d,
            sample['rotation'].numpy(), sample['gt_translation'].numpy(),
            pred_rot.squeeze(0).cpu().numpy(), sample['gt_translation'].numpy()
        )
        results.append(add)

print(f"Mean ADD: {np.mean(results):.4f} m")

100%|██████████| 100/100 [00:01<00:00, 60.41it/s]

Mean ADD: 0.0145 m
